In [1]:
import findspark, random
findspark.init("/u/cs451/packages/spark")

from pyspark import SparkContext, SparkConf
sc = SparkContext(appName="YourTest", master="local[2]", conf=SparkConf().set('spark.ui.port', random.randrange(4000,5000)))

In [2]:
from pyspark.sql import SparkSession
import random
spark = SparkSession.builder.appName("YourTest").master("local[2]").config('spark.ui.port', random.randrange(4000,5000)).getOrCreate()

In [3]:
from pyspark.sql.functions import *
import re
from math import log
from pyspark.sql.functions import udf
from pyspark.sql.types import LongType, StringType
import numpy as np

In [4]:
def lower(line):
    w=[]
    for word in line:
        w.append(word.lower())
    return w

def tokenize(line):
    return re.findall(r"[A-Za-z0-9_-]+",line)

In [5]:
abstract=sc.textFile("abstract.txt").map(lambda l: tokenize(l))
b=abstract.map(lambda l: l[0]).map(lambda x: (x, ))
c=abstract.map(lambda l: lower(l)).map(lambda l: l[1:]).map(lambda x: (x, ))

In [6]:
corpus1=b.toDF().withColumnRenamed( "_1" , "paper_id").withColumn("id", monotonically_increasing_id())
corpus2=c.toDF().withColumnRenamed( "_1" , "documents").withColumn("id", monotonically_increasing_id())
corpus = corpus1.join(corpus2, "id", "outer")

In [7]:
explode=corpus.select(explode("documents"),"paper_id", "documents").withColumnRenamed("col" , "token")

withtf=explode.groupBy("paper_id","token").count().withColumnRenamed("count" , "tf")

withdf=withtf.groupBy("token").agg(countDistinct("paper_id").alias("df"))

In [8]:
num=b.count()+1

In [13]:
withidf1=withdf.withColumn("deno", lit(num).cast(StringType()))

In [14]:
withidf2=withidf1.withColumn("nume", col("df")+1)

In [15]:
withidf3=withidf2.withColumn("idf", log10(col("deno")/col("nume")))

In [16]:
tfidf=withtf.join(withidf3,"token","left").withColumn("tf_idf",col("tf")*col("idf"))

In [34]:
tfidf.orderBy(col("tf_idf").desc()).show()

+--------+--------+---+-----+-------+-----+------------------+------------------+
|   token|paper_id| tf|   df|   deno| nume|               idf|            tf_idf|
+--------+--------+---+-----+-------+-----+------------------+------------------+
| andnbsp|  796734|125|    2|1669283|    3| 5.745408715823863| 718.1760894779828|
|   xlink| 1593453|126|  217|1669283|  218|3.8840734769389207|489.39325809430403|
|    hack| 1342613|121|  225|1669283|  226|3.8684215313961245|468.07900529893107|
|informix|  125180| 99|  106|1669283|  107| 4.193146192858316|415.12147309297325|
|    soho| 2297465| 91|   96|1669283|   97| 4.235758236277281|385.45399950123254|
|     mml| 1927782| 95|  153|1669283|  154| 4.035009249707063|383.32587872217096|
|    thgr|  711390| 93|  132|1669283|  133|4.0986783295764395| 381.1770846506089|
|    nbsp|  882067| 72|    9|1669283|   10| 5.222529970543525|376.02215787913383|
|   xlink| 1680209| 96|  217|1669283|  218|3.8840734769389207| 372.8710537861364|
|ext-link| 15934

In [ ]:
def calcIdf(df):
    return log((D+1)/(df + 1),10)
spark.udf.register("calcIdfWithPython", calcIdf, LongType())

In [ ]:
calcIdf_udf=udf(calcIdf, LongType())
df.select("id", squared_udf("id").alias("id_squared"))

In [ ]:
df=withdf.select("df").rdd.flatMap(lambda x: x).collect()

In [ ]:
df=withdf.select("df").collect().map(_(0)).toList

In [ ]:
withidf=withdf.withColumn("idf", log((D+1)/(col("df") + 1),10))

In [ ]:
explode=spark.sql("SELECT explode(documents) AS token, paper_id, documents \
FROM corpus").cache()
explode.createOrReplaceTempView("explode")

In [ ]:
withtf=spark.sql("SELECT COUNT(documents) AS tf, paper_id, token \
FROM explode \
GROUP BY paper_id, token").cache()
withtf.createOrReplaceTempView("withtf")

In [ ]:
withdf=spark.sql("SELECT COUNT(DISTINCT paper_id) AS DF, token \
FROM withtf \
GROUP BY token").cache()
withdf.createOrReplaceTempView("withdf")

In [ ]:
c=spark.sql("SELECT COUNT(DF) AS count \
FROM withdf")
c.first()["count"]

In [30]:
abstract.filter(lambda x: x[0]=='1618522').collect()

[['1618522',
  'In',
  'dealing',
  'with',
  'the',
  'container',
  'bloat',
  'problem',
  'we',
  'identify',
  'five',
  'memory',
  'compaction',
  'techniques',
  'which',
  'can',
  'be',
  'used',
  'to',
  'reduce',
  'the',
  'footprint',
  'of',
  'the',
  'large',
  'number',
  'of',
  'small',
  'objects',
  'that',
  'make',
  'these',
  'containers',
  'Using',
  'these',
  'techniques',
  'we',
  'describe',
  'two',
  'alternative',
  'methods',
  'for',
  'more',
  'efficient',
  'encoding',
  'of',
  'the',
  'JRE',
  's',
  'ubiquitous',
  'HashMap',
  'data',
  'structure',
  'and',
  'present',
  'a',
  'mathematical',
  'model',
  'in',
  'which',
  'the',
  'footprint',
  'of',
  'this',
  'can',
  'be',
  'analyzed',
  'The',
  'fused',
  'hashing',
  'encoding',
  'method',
  'reduces',
  'memory',
  'overhead',
  'by',
  '20',
  '---45',
  'on',
  'a',
  '32-bit',
  'environment',
  'and',
  '45',
  '---65',
  'on',
  'a',
  '64-bit',
  'environment',
  'Thi